<a href="https://colab.research.google.com/github/nalivaikaanastasiya-dev/Agentic-RAG-Streamlit-App/blob/main/agent_rag_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# agent_rag_pipeline.py
import os
import logging
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.chains import RetrievalQA
from langchain.chat_models import Groq
from langchain.vectorstores import FAISS
from langchain.embeddings import GroqEmbeddings
from tavily import TavilyClient

# -----------------------------
# 1️⃣ Load environment variables
# -----------------------------
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
CHROMA_DB_PATH = os.getenv("CHROMA_DB_PATH", "./faiss_db")

assert GROQ_API_KEY, "GROQ_API_KEY not set"
assert TAVILY_API_KEY, "TAVILY_API_KEY not set"

os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2", "true")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT", "")

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("AgenticRAG")

# -----------------------------
# 2️⃣ Tavily search tool
# -----------------------------
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

def tavily_search(query: str) -> str:
    """Perform a web search using Tavily API"""
    try:
        if not query.strip():
            return "Query is empty."
        logger.info(f"Tavily search query: {query}")
        results = tavily_client.search(query)
        top_results = "\n".join([f"{r['title']}: {r['link']}" for r in results[:3]])
        return top_results
    except Exception as e:
        logger.error(f"Tavily search error: {e}")
        return f"Error: {str(e)}"

# -----------------------------
# 3️⃣ Create or load FAISS vector database
# -----------------------------
def create_vectordb(documents):
    """Create or load a FAISS vector database from documents"""
    try:
        embeddings = GroqEmbeddings(api_key=GROQ_API_KEY)
        if os.path.exists(CHROMA_DB_PATH):
            vectordb = FAISS.load_local(CHROMA_DB_PATH, embeddings)
            logger.info("Loaded existing FAISS DB")
        else:
            texts = [d.page_content if hasattr(d, "page_content") else str(d) for d in documents]
            vectordb = FAISS.from_texts(texts, embeddings)
            vectordb.save_local(CHROMA_DB_PATH)
            logger.info("Created new FAISS DB")
        return vectordb
    except Exception as e:
        logger.error(f"Error creating vector DB: {e}")
        raise e

# -----------------------------
# 4️⃣ Create agent with tools
# -----------------------------
def create_agent(vectordb):
    """Initialize the agent with RAG and Tavily tools"""
    llm = Groq(model="gpt-3.5", api_key=GROQ_API_KEY)
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})
    retrieval_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    tools = [
        Tool(name="RAG", func=retrieval_chain.run, description="Search local knowledge base"),
        Tool(name="TavilySearch", func=tavily_search, description="Search web using Tavily")
    ]
    agent = initialize_agent(tools, llm, agent="structured-chat-zero-shot-react-description", verbose=True)
    return agent

# -----------------------------
# 5️⃣ Python API class
# -----------------------------
class AgenticRAG:
    """Encapsulates the RAG agent for Streamlit usage"""
    def __init__(self, documents):
        self.vectordb = create_vectordb(documents)
        self.agent = create_agent(self.vectordb)

    def get_answer(self, query: str) -> str:
        """Return the agent's answer to a user query"""
        try:
            logger.info(f"User query: {query}")
            answer = self.agent.run(query)
            return answer
        except Exception as e:
            logger.error(f"Agent error: {e}")
            return f"Error: {str(e)}"
